In [111]:
from mesa.time import BaseScheduler
from mesa import Agent, Model
import random
import numpy as np
from scipy.stats import norm

class Scientist(Agent):
    def __init__(self, curr_time, k, means, std_devs, total_self_effort, self_invested_effort, total_ideas, current_idea_effort, max_idea_effort, model):
#         super().__init__(k, means, std_devs, total_self_effort, self_invested_effort, total_ideas, current_idea_effort, max_idea_effort)
        self.self_effort_left = total_self_effort #scalar
        self.self_invested_effort = self_invested_effort
        self.k = k #array
        self.means = means #array
        self.std_devs = std_devs #array
        self.total_ideas = total_ideas
        self.current_idea_effort = current_idea_effort
        self.max_idea_effort = max_idea_effort
        self.time_born = curr_time
    
    def step(self):
        # Pick ideas scientist can work on according to three criteria:
        # 1. the scientist has enough effort left to cover the entry cost "k"
        # 2. the idea has not reached the maximum effort allowed on it
        # 3. the idea was created either in this time period, or the previous time period (CAN BE CHANGED)
        avail = np.nonzero((self.k < self.self_effort_left) & (self.current_idea_effort < self.max_idea_effort))[0]
        
        # Make sure criteria 3 is fulfilled
        idea_time_period = avail//ideas_per_cycle
        crit_3_indices = np.nonzero((idea_time_period == self.time_born) | (idea_time_period == self.time_born - 1))[0]
        avail = avail[crit_3_indices]

        if len(avail) == 0:
            return
        
        avail_idea_effort = self.current_idea_effort[avail]
        avail_means = self.means[avail]
        avail_std_devs = self.std_devs[avail]
        
        # Pick which idea to work on based on highest return
        current_returns = norm.cdf(avail_idea_effort, avail_means, avail_std_devs)
        
        print("current_returns: ", current_returns)
        unit_to_invest = 1
        potential_returns = norm.cdf(avail_idea_effort + unit_to_invest, avail_means, avail_std_devs)
        print("potential_returns: ", potential_returns)
        idea_to_invest_in = np.argmax(potential_returns - current_returns)
        print("idea_to_invest_in: ", idea_to_invest_in)
                
        # Pay the entrance cost to work on the idea "k"
        self.self_effort_left -= self.k[avail[idea_to_invest_in]]
        # Invest in idea with highest potential return (Note: pass by value)
        print("avail[idea_to_invest_in]: ", avail[idea_to_invest_in])
        self.current_idea_effort[avail[idea_to_invest_in]] += unit_to_invest
        print("post_invest_current_idea_effort: ", self.current_idea_effort)
        self.self_invested_effort[avail[idea_to_invest_in]] += unit_to_invest

        
#         # Lessen scientists's total effort by the same amount
#         self.self_effort_left -= unit_to_invest
        
class ScientistModel(Model):
    def __init__(self, scientists_per_cycle, ideas_per_cycle, cycles, granularity):
        self.total_scientists = scientists_per_cycle * cycles
        self.total_ideas = ideas_per_cycle * cycles
        self.granularity = granularity
        self.current_idea_effort = np.zeros(self.total_ideas)
        self.max_idea_effort = 6 * np.ones(self.total_ideas)

        self.schedule = BaseScheduler(self) # has a .time() function
        # Pre-allocate values
        self.k = 1 * np.ones(self.total_ideas)
        self.means = 4 * np.ones(self.total_ideas)
        self.means[2] = 2
        self.std_devs = 1.2 * np.ones(self.total_ideas)
        self.total_self_effort = 5
        self.self_invested_effort = np.zeros(self.total_ideas)

    def step(self):
        for i in range(scientists_per_cycle):
            a = Scientist(self.schedule.time, self.k, self.means, self.std_devs, self.total_self_effort, self.self_invested_effort, self.total_ideas, self.current_idea_effort, self.max_idea_effort, self)
            self.schedule.add(a)
            print("Scientist: ", a.__dict__)
        self.schedule.step()

In [ ]:
cycles = 10
ideas_per_cycle = 10
scientists_per_cycle = 20
granularity = 1
model = ScientistModel(scientists_per_cycle, ideas_per_cycle, cycles, granularity)
for i in range(cycles):
    print("cycle ", i)
    model.step()

In [113]:
# Current Simulation:
# 1. can work on whatever idea regardless of time
#     a. BUT can only invest in one idea per time period
# 2. number of cycle = 10
# 3. will only invest 0.1 units of effort at a time
# 4. everyone gets 1.0 total effort
# 5. 3 scientists and 3 ideas per time period
# 6. maximum effort allowed on each idea = 2
# 7. no Bayesian updating yet

# For the distributions:
#     mean = max_effort/2
#     std_dev = steepness (higher = flatter, lower = steeper)

# Pass in: 
# the true distributions (mean, std_devs) of all the ideas
    # no need for an idea agent
    
# Future conditions for valid ideas to work on
# 1. time period is correct
# 2. we have enough to cover k
# 3. max_effort has not been reached for the idea